In [ ]:

# http://dbpedia.org/page/Autauga_County%2C_Alabama

In [3]:

import sys

# Insert at 1, 0 is the script path (or '' in REPL)
sys.path.insert(1, '../py')

In [2]:

%run ../../load_magic/storage.py
%run ../../load_magic/paths.py
%run ../../load_magic/lists.py
%run ../../load_magic/environment.py
%run ../../load_magic/dataframes.py

import scraping_utils
wikipedia = scraping_utils.wikipedia

from IPython.display import clear_output
from bs4 import BeautifulSoup as bs
from datetime import datetime, timedelta
from matplotlib import pyplot as plt
from scipy import stats
from scipy.stats import pearsonr
from tqdm import tqdm_notebook as tqdm
from urllib import request
from urllib.parse import unquote
import networkx as nx
import numpy as np
import re
import seaborn as sns
import wikidata

notebook_path = get_notebook_path()
print(notebook_path)
s = Storage()
counties_df = s.load_object('counties_df')
print(['s.{}'.format(fn) for fn in dir(s) if not fn.startswith('_')])

%pprint
dir()

D:\Documents\Repositories\notebooks\covid19\ipynb\WikiData Exploration.ipynb
['s.attempt_to_pickle', 's.data_csv_folder', 's.data_folder', 's.encoding_type', 's.load_csv', 's.load_dataframes', 's.load_object', 's.save_dataframes', 's.saves_csv_folder', 's.saves_folder', 's.saves_pickle_folder', 's.store_objects']
Pretty printing has been turned OFF


['Config', 'In', 'Out', 'RandomForestClassifier', 'SequenceMatcher', 'Storage', '_', '__', '___', '__builtin__', '__builtins__', '__doc__', '__loader__', '__name__', '__nonzero__', '__package__', '__spec__', '_dh', '_i', '_i1', '_i2', '_ih', '_ii', '_iii', '_oh', 'bs', 'check_4_doubles', 'check_for_typos', 'clear_output', 'conjunctify_list', 'copyfile', 'counties_df', 'csv', 'datetime', 'encoding', 'example_iterrows', 'exit', 'filepath_regex', 'get_classifier', 'get_column_descriptions', 'get_data_structs_dataframe', 'get_datastructure_prediction', 'get_dir_tree', 'get_git_lfs_track_commands', 'get_importances', 'get_input_sample', 'get_ipython', 'get_max_rsquared_adj', 'get_module_version', 'get_notebook_path', 'get_page_tables', 'get_specific_gitignore_files', 'get_struct_name', 'humanize_bytes', 'io', 'ipykernel', 'json', 'jupyter_config_dir', 'math', 'notebook_path', 'notebookapp', 'np', 'nx', 'os', 'pd', 'pearsonr', 'pickle', 'plt', 'preprocess_data', 'print_all_files_ending_start


# Welcome[¶](#welcome "Permalink to this headline")

[![Build Status](https://img.shields.io/travis/kensho-technologies/qwikidata.svg?style=flat)](https://travis-ci.org/kensho-technologies/qwikidata) [![Documentation Status](https://readthedocs.org/projects/qwikidata/badge/?version=latest)](https://qwikidata.readthedocs.io/en/latest/?badge=latest) [![License](https://img.shields.io/badge/License-Apache%202.0-blue.svg)](https://opensource.org/licenses/Apache-2.0) [![PyPI Python](https://img.shields.io/pypi/pyversions/qwikidata.svg)](https://pypi.python.org/pypi/qwikidata) [![PyPI Version](https://img.shields.io/pypi/v/qwikidata.svg)](https://pypi.python.org/pypi/qwikidata)

<cite>qwikidata</cite> is a Python package with tools that allow you to interact with [Wikidata](https://www.wikidata.org/wiki/Wikidata:Main_Page).

The package defines a set of classes that allow you to represent Wikidata entities in a Pythonic way. It also provides a Pythonic way to access three data sources,

*   [linked data interface](https://www.wikidata.org/wiki/Wikidata:Data_access)
*   [sparql query service](https://www.wikidata.org/wiki/Wikidata:SPARQL_query_service)
*   [json dump](https://www.wikidata.org/wiki/Wikidata:Database_download)


## Quick Install[¶](#quick-install "Permalink to this headline")


### Requirements[¶](#requirements "Permalink to this headline")

*   python >= 3.5


### Install with pip[¶](#install-with-pip "Permalink to this headline")

You can install the most recent version using pip,

In [ ]:

import sys

command_str = '{sys.executable} -m pip install --upgrade qwikidata'.format(sys=sys)
print(command_str)
#!{command_str}


## Quick Examples[¶](#quick-examples "Permalink to this headline")


### Linked Data Interface[¶](#linked-data-interface "Permalink to this headline")

In [ ]:

from qwikidata.entity import WikidataItem, WikidataLexeme, WikidataProperty
from qwikidata.linked_data_interface import get_entity_dict_from_api

# create an item representing "Douglas Adams"
Q_DOUGLAS_ADAMS = "Q42"
q42_dict = get_entity_dict_from_api(Q_DOUGLAS_ADAMS)
q42 = WikidataItem(q42_dict)

# create a property representing "subclass of"
P_SUBCLASS_OF = "P279"
p279_dict = get_entity_dict_from_api(P_SUBCLASS_OF)
p279 = WikidataProperty(p279_dict)

# create a lexeme representing "bank"
L_BANK = "L3354"
l3354_dict = get_entity_dict_from_api(L_BANK)
l3354 = WikidataLexeme(l3354_dict)


### SPARQL Query Service[¶](#sparql-query-service "Permalink to this headline")

In [ ]:

from qwikidata.sparql import (get_subclasses_of_item,
                              return_sparql_query_results)

# send any sparql query to the wikidata query service and get full result back
# here we use an example that counts the number of humans
sparql_query = """
SELECT (COUNT(?item) AS ?count)
WHERE {
 ?item wdt:P31/wdt:P279* wd:Q5 .
}
"""
res = return_sparql_query_results(sparql_query)

# use convenience function to get subclasses of an item as a list of item ids
Q_RIVER = "Q4022"
subclasses_of_river = get_subclasses_of_item(Q_RIVER)


### JSON Dump[¶](#json-dump "Permalink to this headline")

In [1]:

import time

from qwikidata.entity import WikidataItem
from qwikidata.json_dump import WikidataJsonDump
from qwikidata.utils import dump_entities_to_json

P_OCCUPATION = "P106"
Q_POLITICIAN = "Q82955"

def has_occupation_politician(item: WikidataItem, truthy: bool = True) -> bool:
    """Return True if the Wikidata Item has occupation politician."""
    if truthy:
        claim_group = item.get_truthy_claim_group(P_OCCUPATION)
    else:
        claim_group = item.get_claim_group(P_OCCUPATION)

    occupation_qids = [
        claim.mainsnak.datavalue.value["id"]
        for claim in claim_group
        if claim.mainsnak.snaktype == "value"
    ]
    return Q_POLITICIAN in occupation_qids

# create an instance of WikidataJsonDump
wjd_dump_path = "wikidata-20190401-all.json.bz2"
wjd = WikidataJsonDump(wjd_dump_path)

# create an iterable of WikidataItem representing politicians
politicians = []
t1 = time.time()
for ii, entity_dict in enumerate(wjd):

    if entity_dict["type"] == "item":
        entity = WikidataItem(entity_dict)
        if has_occupation_politician(entity):
            politicians.append(entity)

    if ii % 1000 == 0:
        t2 = time.time()
        dt = t2 - t1
        print(
            "found {} politicians among {} entities [entities/s: {:.2f}]".format(
                len(politicians), ii, ii / dt
            )
        )

    if ii > 10000:
        break

# write the iterable of WikidataItem to disk as JSON
out_fname = "filtered_entities.json"
dump_entities_to_json(politicians, out_fname)
wjd_filtered = WikidataJsonDump(out_fname)

# load filtered entities and create instances of WikidataItem
for ii, entity_dict in enumerate(wjd_filtered):
    item = WikidataItem(entity_dict)

ModuleNotFoundError: No module named 'qwikidata'


## License[¶](#license "Permalink to this headline")

Licensed under the Apache 2.0 License. Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an “AS IS” BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.


## Copyright[¶](#copyright "Permalink to this headline")

Copyright 2019 Kensho Technologies, LLC.


## Important Links[¶](#important-links "Permalink to this headline")


[readthedocs](https://qwikidata.readthedocs.io/en/stable/readme.html) | [PyPI](https://pypi.org/project/qwikidata/) | [github](https://github.com/kensho-technologies/qwikidata/)


----

In [396]:

import pyperclip

pyperclip.copy(str(tuple((v, k) for k, v in idx_dict.items())))